In [1]:
!pip install xgboost

You should consider upgrading via the 'C:\Users\hhong\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [83]:
#quick random forest regression for predicting wins
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

In [3]:
#we try to find the difference in score home_team_score - away_team_score
complete_matchDf = pd.read_pickle('complete_match.pkl')

In [4]:
complete_matchDf.head()

match_id  home_team_id  away_team_id                        date  \
0    665626          8635          8342  2010-02-03 00:00:00.000000   
1    665630          9986          9985  2010-02-04 00:00:00.000000   
2    665634          8203          9993  2010-02-02 00:00:00.000000   
3    665665          8342         10001  2010-01-30 00:00:00.000000   
4    665666          9985          8635  2010-01-17 00:00:00.000000   

   home_team_goal  away_team_goal  home_team_build_up_play_speed  \
0               3               2                             50   
1               2               3                             40   
2               1               0                             65   
3               2               1                             35   
4               0               4                             53   

   home_team_build_up_play_dribbling  home_team_build_up_play_passing  \
0                          48.607362                               35   
1                          48.607362                               50   
2                          48.607362                               60   
3                          48.607362                               40   
4                          48.607362                               40   

   home_team_chance_creationg_passing  ...  home_team_defence_team_width  \
0                                  70  ...                            70   
1                                  45  ...                            70   
2                                  50  ...                            60   
3                                  45  ...                            70   
4                                  55  ...                            65   

   away_team_build_up_play_speed  away_team_build_up_play_dribbling  \
0                             35                          48.607362   
1                             53                          48.607362   
2                             35                          48.607362   
3                             45                          48.607362   
4                             50                          48.607362   

   away_team_build_up_play_passing  away_team_chance_creationg_passing  \
0                               40                                  45   
1                               40                                  55   
2                               35                                  45   
3                               60                                  40   
4                               35                                  70   

   away_team_chance_creation_crossing  away_team_chance_creation_shooting  \
0                                  50                                  45   
1                                  55                                  65   
2                                  40                                  50   
3                                  45                                  55   
4                                  50                                  60   

   away_team_defence_pressure  away_team_defence_aggression  \
0                          60                            70   
1                          70                            70   
2                          70                            70   
3                          70                            70   
4                          70                            50   

   away_team_defence_team_width  
0                            70  
1                            65  
2                            70  
3                            70  
4                            70  

[5 rows x 24 columns]

In [5]:
 complete_matchDf['result'] = complete_matchDf['home_team_goal'] - complete_matchDf['away_team_goal']

In [6]:
complete_matchDf['result']

0        1
1       -1
2        1
3        1
4       -4
        ..
18238    0
18239    1
18240   -1
18241    2
18242    1
Name: result, Length: 18243, dtype: int64

In [7]:
y = complete_matchDf.result
features_of_interest = [
    'home_team_build_up_play_speed',
    'home_team_build_up_play_dribbling',
    'home_team_build_up_play_passing',
    'home_team_chance_creationg_passing',
    'home_team_chance_creation_crossing',
    'home_team_chance_creation_shooting',
    'home_team_defence_pressure',
    'home_team_defence_aggression',
    'home_team_defence_team_width',
    'away_team_build_up_play_speed',
    'away_team_build_up_play_dribbling',
    'away_team_build_up_play_passing',
    'away_team_chance_creationg_passing',
    'away_team_chance_creation_crossing',
    'away_team_chance_creation_shooting',
    'away_team_defence_pressure',
    'away_team_defence_aggression',
    'away_team_defence_team_width',
]
X = complete_matchDf[features_of_interest]

In [8]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Define model
decision_tree = DecisionTreeRegressor()
# Fit model
decision_tree.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = decision_tree.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

1.825032531599074


In [9]:
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
prediction = forest_model.predict(val_X)
print(mean_absolute_error(val_y, prediction))
#mean absolute error of 1.4060983398862479. Not bad?

1.4060983398862479


In [84]:
minimum = 1000
bestIndex = -1
#you can't have 0 as n_estimator
for ii in range(1, 100):
    XG_model = XGBRegressor(n_estimators= ii)
    XG_model.fit(train_X, train_y, 
                 early_stopping_rounds=5, 
                 eval_set=[(val_X, val_y)],
                 verbose=False)
    predictions = np.rint(XG_model.predict(val_X))
    MAE = mean_absolute_error(val_y, predictions)
    if MAE < minimum:
        minimum = MAE
        bestIndex = ii

print(bestIndex)
print(minimum)

7
1.3014689761017322


In [87]:
XG_model = XGBRegressor(n_estimator = 7)
XG_model.fit(train_X, train_y, 
                 early_stopping_rounds=5, 
                 eval_set=[(val_X, val_y)],
                 verbose=False)
predictions = XG_model.predict(val_X)
ultimate_predictions = np.rint(predictions)
total = len(ultimate_predictions)
just_win_loss_tie = np.sign(val_y)
just_win_loss_tie_predictions = np.sign(ultimate_predictions)
correct = np.sum(just_win_loss_tie == just_win_loss_tie_predictions)
print(correct/total)
#so this is terrible




[23:38:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0.39026529269896953


In [25]:
#best mean_absolute_error at around:
#1.3262471103714237

In [33]:
reg = LinearRegression().fit(train_X, train_y)

In [35]:
mean_absolute_error(val_y, reg.predict(val_X))

1.3628156770705915

In [68]:
minimum = 1000
bestIndex = -1

for ii in range(1, 100):
    ridge_reg = Ridge(alpha = ii)
    ridge_reg.fit(train_X, train_y)
    predictions = ridge_reg.predict(val_X)
    MAE = mean_absolute_error(val_y, predictions)
    if MAE < minimum:
        minimum = MAE
        bestIndex = ii

print(bestIndex)
print(minimum) 
    

1
1.3628156772632816


In [73]:
minimum = 1000
bestIndex = -1

for ii in range(1, 100):
    lasso_reg = Lasso(alpha = ii)
    lasso_reg.fit(train_X, train_y)
    predictions = lasso_reg.predict(val_X)
    MAE = mean_absolute_error(val_y, predictions)
    if MAE < minimum:
        minimum = MAE
        bestIndex = ii

print(bestIndex)
print(minimum)    

1
1.3806849231284506


In [78]:
pcr = make_pipeline(StandardScaler(), PCA(n_components=1), LinearRegression())
pcr.fit(train_X, train_y)
pca = pcr.named_steps['pca']  # retrieve the PCA step of the pipeline
print(f"PCR r-squared {pcr.score(val_X, val_y):.3f}")

PCR r-squared -0.000


In [79]:
pls = PLSRegression(n_components=1)
pls.fit(train_X, train_y)
print(f"PLS r-squared {pls.score(val_X, val_y):.3f}")

PLS r-squared 0.024
